In [ ]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
from bokeh.sampledata.autompg import autompg_clean as df
import seaborn as sns

# Initialize Panel extension
pn.extension()

# Get numeric and categorical columns
numeric_cols = list(df.select_dtypes(include=[np.number]).columns)
categorical_cols = list(df.select_dtypes(exclude=[np.number]).columns)

# Create widgets
select_var = pn.widgets.Select(
    options=numeric_cols,
    name='Variable',
    value='mpg'
)

select_group = pn.widgets.Select(
    options=['origin', 'mfr'],
    name='Group By',
    value='origin'
)

# Define plotting functions
@pn.depends(select_var, select_group)
def histogram_plot(select_var, select_group):
    return df.hvplot.hist(
        y=select_var,
        by=select_group,
        bins=20,
        height=300,
        title=f'Histogram of {select_var}',
        alpha=0.6
    )

@pn.depends(select_var, select_group)
def box_plot(select_var, select_group):
    return df.hvplot.box(
        y=select_var,
        by=select_group,
        height=300,
        title=f'Box Plot of {select_var} by {select_group}'
    )

@pn.depends(select_var)
def scatter_plot(select_var):
    return df.hvplot.scatter(
        x=select_var,
        y='mpg' if select_var != 'mpg' else 'hp',
        by='origin',
        height=400,
        width=400,
        title=f'Scatter Plot: {select_var} vs {"MPG" if select_var != "mpg" else "Horsepower"}'
    )

@pn.depends(select_var, select_group)
def summary_stats(select_var, select_group):
    stats = df.groupby(select_group)[select_var].agg([
        'count', 'mean', 'std', 'min', 'max'
    ]).round(2)

    return pn.pane.DataFrame(
        stats,
        width=400,
        height=200,
        name=f'Summary Statistics for {select_var}'
    )

# Create correlation heatmap
def correlation_plot():
    corr_data = df[numeric_cols].corr().round(2)
    return df[numeric_cols].hvplot.heatmap(
        title='Correlation Matrix',
        height=400,
        width=400,
        xaxis='top'
    )

# Create the dashboard layout
dashboard = pn.Column(
    '# Interactive EDA Dashboard',
    pn.Row(select_var, select_group),
    pn.Row(
        pn.Column(histogram_plot),
        pn.Column(box_plot)
    ),
    pn.Row(
        pn.Column(scatter_plot),
        pn.Column(correlation_plot())
    ),
    pn.Row(summary_stats)
)

# Display the dashboard
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Select(name='Variable', options=['mpg', 'cyl', ...], value='mpg')
        [1] Select(name='Group By', options=['origin', 'mfr'], value='origin')
    [2] Row
        [0] Column
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
        [1] Column
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [3] Row
        [0] Column
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
        [1] Column
            [0] HoloViews(HeatMap, height=400, sizing_mode='fixed', width=400)
    [4] Row
        [0] ParamFunction(function, _pane=DataFrame, defer_load=False)